In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path
from tqdm import tqdm

def process_and_visualize_split(split_name, base_dataset_dir, model, base_output_dir):
    """
    Memproses semua gambar dalam satu set data (train, val, atau test)
    dan menyimpan visualisasi perbandingannya. Jika visualisasi sudah ada,
    proses akan dilewati.

    Args:
        split_name (str): Nama set data ('train', 'val', 'test').
        base_dataset_dir (Path): Path objek ke direktori dasar dataset.
        model (YOLO): Objek model YOLO yang sudah dimuat.
        base_output_dir (Path): Path objek ke direktori dasar output.
    """
    print(f"\n{'='*25}\nProcessing split: {split_name.upper()}\n{'='*25}")

    # 1. Definisikan semua path yang relevan untuk split ini
    image_dir = base_dataset_dir / 'maskDaun' / split_name / 'images'
    mask_label_dir = base_dataset_dir / 'maskDaun' / split_name / 'labels'
    bbox_label_dir = base_dataset_dir / 'objectDetection' / split_name / 'labels'
    output_dir = base_output_dir / split_name

    output_dir.mkdir(parents=True, exist_ok=True)

    image_files = list(image_dir.glob('*.jpg')) + list(image_dir.glob('*.jpeg')) + list(image_dir.glob('*.png'))

    if not image_files:
        print(f"⚠️  Peringatan: Tidak ada gambar ditemukan di {image_dir}, melewati split ini.")
        return

    # 2. Loop melalui setiap gambar dalam direktori dengan progress bar
    for image_path in tqdm(image_files, desc=f"Processing {split_name}"):
        base_name = image_path.stem
        
        # --- MODIFIKASI DIMULAI DI SINI ---
        
        # Tentukan path output terlebih dahulu
        output_filename = f"komparasi_{base_name}.png"
        output_path = output_dir / output_filename

        # Jika file output sudah ada, lewati (skip) proses untuk gambar ini
        if output_path.exists():
            continue # Lanjut ke file berikutnya di dalam loop

        # --- AKHIR MODIFIKASI ---

        image = cv2.imread(str(image_path))
        if image is None:
            tqdm.write(f"❌ Gagal membaca {image_path.name}, dilewati.")
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        h, w, _ = image.shape
        
        image_with_bbox = image_rgb.copy()
        image_for_mask = image_rgb.copy()

        # 3. Gambar Ground Truth Bounding Box
        bbox_label_path = bbox_label_dir / f"{base_name}.txt"
        if bbox_label_path.exists():
            with open(bbox_label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    _, x_c, y_c, w_norm, h_norm = map(float, parts)
                    x1, y1 = int((x_c - w_norm / 2) * w), int((y_c - h_norm / 2) * h)
                    x2, y2 = int((x_c + w_norm / 2) * w), int((y_c + h_norm / 2) * h)
                    cv2.rectangle(image_with_bbox, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2)
        
        # 4. Gambar Ground Truth Mask
        mask_label_path = mask_label_dir / f"{base_name}.txt"
        if mask_label_path.exists():
            mask_overlay = image_for_mask.copy()
            with open(mask_label_path, 'r') as f:
                for line in f:
                    polygon_norm = np.array(line.strip().split()[1:], dtype=np.float32).reshape(-1, 2)
                    polygon_pixels = (polygon_norm * np.array([w, h])).astype(np.int32)
                    cv2.fillPoly(mask_overlay, [polygon_pixels], color=(0, 255, 0))
            image_with_mask = cv2.addWeighted(mask_overlay, 0.5, image_for_mask, 0.5, 0)
        else:
            image_with_mask = image_for_mask

        # 5. Lakukan Prediksi dengan Model (hanya jika file output belum ada)
        results = model.predict(source=str(image_path))
        predicted_image_bgr = results[0].plot()
        predicted_image_rgb = cv2.cvtColor(predicted_image_bgr, cv2.COLOR_BGR2RGB)

        # 6. Buat dan simpan plot 3 panel
        fig, axes = plt.subplots(1, 3, figsize=(24, 8))
        fig.suptitle(f"Perbandingan Anotasi untuk: {image_path.name}", fontsize=16)

        axes[0].imshow(image_with_bbox)
        axes[0].set_title("Ground Truth (Bounding Box)")
        axes[0].axis('off')

        axes[1].imshow(image_with_mask)
        axes[1].set_title("Ground Truth (Mask)")
        axes[1].axis('off')
        
        axes[2].imshow(predicted_image_rgb)
        axes[2].set_title("Hasil Prediksi Model")
        axes[2].axis('off')

        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        
        # Simpan file ke path yang telah ditentukan di awal loop
        plt.savefig(output_path)
        plt.close(fig)

    print(f"✅  Selesai memproses split {split_name}. Hasil disimpan di: {output_dir}")

# --- Main Execution Block ---
if __name__ == "__main__":
    # Tentukan path utama. Gunakan Path() untuk konsistensi antar OS.
    # Diasumsikan skrip ini ada di folder di level yang sama dengan 'dataset' dan 'results'.
    BASE_DATASET_DIR = Path('dataset')
    MODEL_PATH = Path('best.pt')
    BASE_OUTPUT_DIR = Path('view/maskDaun')

    # Periksa keberadaan model sebelum memulai
    if not MODEL_PATH.exists():
        print(f"❌ KESALAHAN FATAL: File model tidak ditemukan di {MODEL_PATH}")
        print("Pastikan path model sudah benar sebelum menjalankan skrip.")
    else:
        print("Memuat model YOLO. Mohon tunggu...")
        yolo_model = YOLO(MODEL_PATH)
        print("Model berhasil dimuat.")

        # splits_to_process = ['train']
        splits_to_process = ['test']
        # splits_to_process = ['valid']
        for split in splits_to_process:
            process_and_visualize_split(split, BASE_DATASET_DIR, yolo_model, BASE_OUTPUT_DIR)

        print("\n🎉 Semua proses telah selesai!")

Memuat model YOLO. Mohon tunggu...
Model berhasil dimuat.

Processing split: TEST


Processing test:  89%|████████▉ | 1549/1739 [00:00<00:00, 4918.66it/s]


image 1/1 /workspace/dataset/maskDaun/test/images/healthy_test_199_jpg.rf.5d8f75739aa067efb60f872844ecfbe4.jpg: 448x640 3 leaf_healthys, 50.6ms
Speed: 5.7ms preprocess, 50.6ms inference, 163.0ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /workspace/dataset/maskDaun/test/images/healthy_test_198_jpg.rf.af9f91fcec183a7ac30b9d98b59f2de8.jpg: 448x640 1 leaf_scab, 9.2ms
Speed: 1.6ms preprocess, 9.2ms inference, 2.1ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /workspace/dataset/maskDaun/test/images/healthy_test_197_jpg.rf.edfa20d7ce39da81e86889fc8d864dae.jpg: 448x640 4 leaf_healthys, 9.9ms
Speed: 1.7ms preprocess, 9.9ms inference, 2.7ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /workspace/dataset/maskDaun/test/images/healthy_test_195_jpg.rf.927a1dfbef83c5a37d9741eb4e4efca8.jpg: 448x640 2 leaf_healthys, 9.6ms
Speed: 1.7ms preprocess, 9.6ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /workspace/dataset/maskDaun/t

Processing test:  95%|█████████▍| 1647/1739 [00:20<00:02, 43.74it/s]  


image 1/1 /workspace/dataset/maskDaun/test/images/healthy_test_183_jpg.rf.b0845e2bcbe175c75be5bb2759eeb04e.jpg: 448x640 4 leaf_healthys, 10.5ms
Speed: 1.7ms preprocess, 10.5ms inference, 1.7ms postprocess per image at shape (1, 3, 448, 640)


...


image 1/1 /workspace/dataset/maskDaun/test/images/healthy_test_101_jpg.rf.e3d192646352805ac0c1dd7b5a03237e.jpg: 448x640 2 leaf_healthys, 9.7ms
Speed: 1.6ms preprocess, 9.7ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)


Processing test: 100%|█████████▉| 1738/1739 [02:02<00:00,  1.50it/s]


image 1/1 /workspace/dataset/maskDaun/test/images/healthy_test_100_jpg.rf.e1a62c8829771ce4a544178acf5b9645.jpg: 448x640 1 leaf_healthy, 1 leaf_rust, 1 leaf_scab, 10.9ms
Speed: 1.8ms preprocess, 10.9ms inference, 1.8ms postprocess per image at shape (1, 3, 448, 640)


Processing test: 100%|██████████| 1739/1739 [02:03<00:00, 14.10it/s]

✅  Selesai memproses split test. Hasil disimpan di: view/maskDaun/test

🎉 Semua proses telah selesai!


In [3]:
import torch
if torch.cuda.is_available():
    device = 'cuda'
    gpu_count = torch.cuda.device_count()
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU terdeteksi! Menggunakan '{device}'.")
    print(f"   - Jumlah GPU: {gpu_count}")
    print(f"   - Nama GPU: {gpu_name}")
else:
    device = 'cpu'
    print("⚠️ Peringatan: GPU tidak ditemukan. Menggunakan 'cpu'.")
    print("   Proses prediksi akan berjalan jauh lebih lambat.")

✅ GPU terdeteksi! Menggunakan 'cuda'.
   - Jumlah GPU: 1
   - Nama GPU: NVIDIA GeForce RTX 4090
